In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

file_path = r"D:\USERLOG"
save_path = os.path.join(file_path, 'select_user')

pd.set_option('display.max_columns', 15)

### Raw data

In [2]:
userset = pickle.load(open(os.path.join(file_path, 'userset10.pkl'), 'rb'))
userset.head(10)

,ID,InTime,OutTime,CH,OnOff_sess
0,21672153690,2017-03-19 23:57:37,2017-03-19 23:59:53,S15,1
1,21672153690,2017-03-20 08:39:43,2017-03-20 08:39:55,S53,2
2,21672153690,2017-03-20 08:39:55,2017-03-20 08:40:05,Jap,2
2,21672153690,2017-03-20 08:40:05,2017-03-20 08:40:17,S65,2
3,21672153690,2017-03-20 08:40:17,2017-03-20 08:42:43,S66,2
4,21672153690,2017-03-20 08:42:43,2017-03-20 08:43:01,Jap,2
4,21672153690,2017-03-20 08:43:01,2017-03-20 08:43:11,S73,2
5,21672153690,2017-03-20 08:43:11,2017-03-20 08:43:21,Jap,2
5,21672153690,2017-03-20 08:43:21,2017-03-20 08:45:50,S76,2
6,21672153690,2017-03-20 08:45:50,2017-03-20 08:45:52,Jap,2


#### Removed 3/27

In [ ]:
drop_day = [i for i in range(len(userset)) if userset.iloc[i].OutTime.day == 27]

In [ ]:
# add time
def bet_second(dataset, timeind):
    sec = int(dataset.OutTime[timeind].value) - int(dataset.InTime[timeind].value)
    sec_ = sec / 1000000000
    return int(sec_)

bets = [bet_second(select_day, ind) for ind in range(len(select_day))]
userset['bets'] = bets

### User selection

In [4]:
userset = pickle.load(open(os.path.join(save_path, 'user10_dayselect_except27.pkl'), 'rb'))
userset.head(10)

,ID,InTime,OutTime,CH,OnOff_sess,bets
0,21672153690,2017-03-19 23:57:37,2017-03-19 23:59:53,S15,1,136
1,21672153690,2017-03-20 08:39:43,2017-03-20 08:39:55,S53,2,12
2,21672153690,2017-03-20 08:39:55,2017-03-20 08:40:05,Jap,2,10
3,21672153690,2017-03-20 08:40:05,2017-03-20 08:40:17,S65,2,12
4,21672153690,2017-03-20 08:40:17,2017-03-20 08:42:43,S66,2,146
5,21672153690,2017-03-20 08:42:43,2017-03-20 08:43:01,Jap,2,18
6,21672153690,2017-03-20 08:43:01,2017-03-20 08:43:11,S73,2,10
7,21672153690,2017-03-20 08:43:11,2017-03-20 08:43:21,Jap,2,10
8,21672153690,2017-03-20 08:43:21,2017-03-20 08:45:50,S76,2,149
9,21672153690,2017-03-20 08:45:50,2017-03-20 08:45:52,Jap,2,2


In [12]:
# Total viewing time per ID, session.
sess_sum_df = userset.groupby(["ID", "OnOff_sess"]).sum().reset_index()
sess_sum_df.head(13)

,ID,OnOff_sess,bets
0,21672153690,1,136
1,21672153690,2,2911
2,21672153690,3,13220
3,21672153690,4,7035
4,21672153690,5,27763
5,21672153690,6,14477
6,21672153690,7,14955
7,21672153690,8,12606
8,21672153690,9,2716
9,21672153690,10,3610


In [14]:
# Remove less than 60s
sess_sum_df = sess_sum_df[sess_sum_df.bets >= 60]

In [15]:
# Total viewing time per ID.
sess_sum_df = sess_sum_df.groupby(["ID"]).sum().reset_index()
sess_sum_df[sess_sum_df.bets < 100].head(10)

# Remove less than 100s
sess_sum_df = sess_sum_df[sess_sum_df.bets >= 100]

,ID,OnOff_sess,bets
683,21693022160,34825,82
8463,21916941780,368760,97
10281,21970388140,447278,91
12273,22023919210,533286,77
13443,22063571730,583636,92
19393,22256585050,846289,62
20031,22269640190,872914,74
20799,22289203900,907290,89
21353,22301463150,930901,71
24368,22387262670,1059978,88


In [21]:
CH_m = []
for i in tqdm(range(len(userset))):
    if select_day_user.bets[i] < 60:
        CH_m.append('Jap')
    else: CH_m.append(userset.CH[i])
userset['CH_m'] = CH_m

userset.head(10)

,ID,InTime,OutTime,CH,OnOff_sess,bets,day,CH_m
0,04,2017-03-19 23:17:09,2017-03-20 00:13:07,S13,97,3358,20,S13
1,04,2017-03-20 00:13:07,2017-03-20 00:13:38,Jap,97,31,20,Jap
2,04,2017-03-20 00:13:38,2017-03-20 00:15:44,S25,97,126,20,S25
3,04,2017-03-20 00:15:44,2017-03-20 00:17:06,Jap,97,82,20,Jap
4,04,2017-03-20 00:17:06,2017-03-20 00:25:35,S19,97,509,20,S19
5,04,2017-03-20 00:25:35,2017-03-20 00:25:49,Jap,97,14,20,Jap
6,04,2017-03-20 00:25:49,2017-03-20 00:26:09,S69,97,20,20,Jap
7,04,2017-03-20 00:26:09,2017-03-20 00:26:34,S70,97,25,20,Jap
8,04,2017-03-20 00:26:34,2017-03-20 00:28:47,S71,97,133,20,S71
9,04,2017-03-20 00:28:47,2017-03-20 00:28:48,Jap,97,1,20,Jap


### Channel Matching

In [23]:
matching.head(10)

,ID,InTime,OutTime,CH,OnOff_sess,bets,day,CH_m,CH_match
0,04,2017-03-19 23:17:09,2017-03-20 00:13:07,S13,97,3358,20,S13,S13
1,04,2017-03-20 00:13:07,2017-03-20 00:13:38,Jap,97,31,20,Jap,Jap
2,04,2017-03-20 00:13:38,2017-03-20 00:15:44,S25,97,126,20,S25,S25
3,04,2017-03-20 00:15:44,2017-03-20 00:17:06,Jap,97,82,20,Jap,Jap
4,04,2017-03-20 00:17:06,2017-03-20 00:25:35,S19,97,509,20,S19,S20
5,04,2017-03-20 00:25:35,2017-03-20 00:25:49,Jap,97,14,20,Jap,Jap
6,04,2017-03-20 00:25:49,2017-03-20 00:26:09,S69,97,20,20,Jap,Jap
7,04,2017-03-20 00:26:09,2017-03-20 00:26:34,S70,97,25,20,Jap,Jap
8,04,2017-03-20 00:26:34,2017-03-20 00:28:47,S71,97,133,20,S71,S66
9,04,2017-03-20 00:28:47,2017-03-20 00:28:48,Jap,97,1,20,Jap,Jap
